In [3]:
import torch
import torchvision.transforms as transforms
import torchvision
from torch import nn,optim
import torch.nn.functional as F
import time

In [4]:
import numpy as np

In [5]:
a=np.array([[1,2,3],[4,5,6]])
np.sum(a,axis=0,keepdims=True)

array([[5, 7, 9]])

- 多少个输出通道,就要分别每个通道做归一化,所以输出通道保持不变
- 训练与预测不同,需要需要通过均值,方差归一化,预测要用移动平均方法
- 2是全连接层,4是卷积层

In [12]:
def batch_norm(is_training,X,gamma,beta,moving_mean,moving_var,eps,momentum):
    if not is_training:
        X_hat=(X-moving_mean) / torch.sqrt(moving_var+eps)
    else:
        assert len(X.shape) in (2,4)
        if len(X.shape)==2:
            mean=X.mean(dim=0)
            var=((X-mean)**2).mean(dim=0)
        else:
            mean=X.mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
            var=((X-mean)**2).mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
        X_hat=(X-mean)/torch.sqrt(var+eps)
        moving_mean=momentum*moving_mean+(1.0-momentum)*mean
        moving_var=momentum*moving_var+(1.0-momentum)*var
    Y=gamma*X_hat+beta
    return Y,moving_mean,moving_var

- BatchNorm 实例
所需指定的 num_features 参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数

In [13]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        ## nn.Parameter 是一种类型
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
                                                          X,
                                                          self.gamma,
                                                          self.beta,
                                                          self.moving_mean,
                                                          self.moving_var,
                                                          eps=1e-5,
                                                          momentum=0.9)
        return Y

In [20]:
class FlattenLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x.view(x.size()[0],-1)

In [21]:
net = nn.Sequential(
    nn.Conv2d(1, 6, 5),
    BatchNorm(6, num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),
    nn.Conv2d(6,16,5),
    BatchNorm(16,num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),
    FlattenLayer(),
    nn.Linear(16*4*4,120),
    BatchNorm(120,num_dims=2),
    nn.Sigmoid(),
    nn.Linear(120,84),
    BatchNorm(84,num_dims=2),
    nn.Sigmoid(),
    nn.Linear(84,10)
)

In [22]:
def load_data_fashion_mnist(batch_size,
                            resize=None,
                            root=r'F:\study\ml\DataSet\FashionMNIST'):
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root,
                                                    train=True,
                                                    download=True,
                                                    transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root,
                                                   train=False,
                                                   download=True,
                                                   transform=transform)

    train_iter = torch.utils.data.DataLoader(mnist_train,
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=4)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                            batch_size=batch_size,
                                            shuffle=False,
                                            num_workers=4)
    return train_iter, test_iter

In [23]:
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, num_epochs):
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc, n, start = 0.0, 0.0, 0, time.time()
        for x, y in train_iter:
            y_hat = net(x)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            n += y.shape[0]
            batch_count += 1
            train_acc += (y_hat.argmax(dim=1)==y).sum().item()
#             break
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d,loss %.4f,train acc %.3f,test acc %.3f,time %.1f sec' %
              (epoch + 1, train_l_sum / batch_count, train_acc / n, test_acc,
               time.time() - start))

In [24]:
def evaluate_accuracy(data_iter,net):
    acc_sum,n=0.0,0
    with torch.no_grad():
        for x,y in data_iter:
            net.eval()
            acc_sum +=(net(x).argmax(dim=1)==y).float().sum().item()
            net.train()
            n+=y.shape[0]
        return acc_sum /n

In [25]:
batch_size=256
train_iter,test_iter=load_data_fashion_mnist(batch_size=batch_size)

lr,num_epochs=0.001,5
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
train_ch5(net,train_iter,test_iter,batch_size,optimizer,num_epochs)


epoch 1,loss 1.0109,train acc 0.781,test acc 0.820,time 35.6 sec
epoch 2,loss 0.2342,train acc 0.861,test acc 0.834,time 33.4 sec
epoch 3,loss 0.1246,train acc 0.876,test acc 0.847,time 32.2 sec
epoch 4,loss 0.0846,train acc 0.883,test acc 0.847,time 32.2 sec
epoch 5,loss 0.0630,train acc 0.891,test acc 0.751,time 32.1 sec


In [30]:
net[1].gamma.view((-1,)),net[1].beta.view((-1,))

(tensor([1.0270, 0.9866, 1.2438, 1.0900, 0.9440, 0.8653],
        grad_fn=<ViewBackward>),
 tensor([-0.6213, -0.6319,  0.1360, -0.7276, -0.1986, -0.2034],
        grad_fn=<ViewBackward>))